In [3]:
import numpy as np

import pandas as pd
# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

import kit as k    

In [4]:
df = k.get_hfi_returns()
df = df["2000":]

df.head(3)

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
date,,,,,,,,,,,,,
2000-01,0.0227,0.0128,0.0088,0.0077,0.0075,0.0088,0.0041,0.0021,0.0075,0.0143,0.0173,0.0427,0.0169
2000-02,0.0267,-0.0022,0.0421,0.0528,0.0253,0.0346,0.0097,0.0408,0.0699,0.0239,0.0185,-0.1340,0.0666
2000-03,0.0243,-0.0138,0.0103,0.0318,0.0134,0.0069,-0.0061,-0.0104,0.0006,0.0131,0.0163,-0.0230,0.0039


In [25]:
ind = k.get_ind_returns()
ind.head(3)

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,Cnstr,Steel,FabPr,ElcEq,Autos,Carry,Mines,Coal,Oil,Util,Telcm,Servs,BusEq,Paper,Trans,Whlsl,Rtail,Meals,Fin,Other
1926-07,0.0056,-0.0519,0.0129,0.0293,0.1097,-0.0048,0.0808,0.0177,0.0814,0.0039,0.0207,0.0407,0.0543,0.0318,0.1639,0.0102,0.0564,0.0154,-0.0140,0.0704,0.0083,0.0922,0.0206,0.0770,0.0193,-0.2379,0.0007,0.0187,0.0037,0.0520
1926-08,0.0259,0.2703,0.0650,0.0055,0.1001,-0.0358,-0.0251,0.0425,0.0550,0.0814,0.0472,0.0217,0.0206,0.0210,0.0423,0.0166,0.0055,0.0085,0.0369,-0.0169,0.0217,0.0202,0.0439,-0.0238,0.0488,0.0539,-0.0075,-0.0013,0.0446,0.0676
1926-09,0.0116,0.0402,0.0126,0.0658,-0.0099,0.0073,-0.0051,0.0069,0.0533,0.0231,-0.0050,0.0015,0.0036,-0.0056,0.0483,0.0273,0.0174,0.0030,-0.0369,0.0204,0.0241,0.0225,0.0019,-0.0554,0.0005,-0.0787,0.0025,-0.0056,-0.0123,-0.0386


---

In [6]:
#1
var = k.var_gaussian(df['Distressed Securities'], 1, modified= False)
print( f'{var*100 :.2f}' )

3.14


In [7]:
#2
var = k.var_gaussian(df['Distressed Securities'], 1, modified= True)
print( f'{var*100 :.2f}' )

4.97


In [8]:
#3
var = k.var_historic(df['Distressed Securities'], 1)
print( f'{var*100 :.2f}' )

4.26


In [9]:
#5
sectors = ['Books', 'Steel', 'Oil', 'Mines']
ind = ind['2013':'2017'][sectors]

er = k.annualize_rets(ind)
cov = ind.cov()

w1 = k.msr(0.1, er, cov)
max = np.max(w1)

print( f'{max*100 :.2f}' )

100.00


In [10]:
#6
indx = np.where( w1 == max )[0][0]
print(sectors[indx])

Steel


In [11]:
#7
elems = w1[w1 > 0.0000000001]
print(len(elems))

1


In [12]:
#8
w2 = k.gmv(cov)
max = np.max(w2)

print( f'{max*100 :.2f}' )

47.70


In [13]:
#9
indx = np.where( w2 == max )[0][0]
print(sectors[indx])

Books


In [14]:
#10
elems = w2[w2 > 0.0000000001]
print(len(elems))

3


In [15]:
#11
ind = k.get_ind_returns()
ind = ind.loc['2018', sectors]

cov = ind.cov()

vol = k.portfolio_vol(w1, cov)
vol *= 12**0.5

print( f'{vol*100 :.2f}' )

21.98


In [16]:
#12
vol = k.portfolio_vol(w2, cov)
vol *= 12**0.5

print( f'{vol*100 :.2f}' )

18.97


---

For-Loops in python are expensive, and we have so far relied on numpy to do the looping for us efficiently in C. The problem is that it's easy to miss the fact that there is a loop here.

In [17]:
def compound1(r):
    ''' 
    expm1:  e^{x-1}
    log1p: ln(x+1)

    ln(x1 + 1) + ln(x2 + 1): ln[ (x1+1)(x2+1) ]
    '''
    return np.expm1(np.log1p(r).sum())

def compound2(r):
    return (r+1).prod()-1

In [26]:
%timeit compound1(ind[["Steel", "Fin", "Beer"]])

605 µs ± 5.98 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
%timeit compound2(ind[["Steel", "Fin", "Beer"]])

569 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


---